# Experiment B

In [1]:
import os
import sys


import pandas as pd

from tqdm import tqdm

# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# Add the parent directory to the system path
sys.path.append(os.path.join(notebook_dir, '../utils'))

from llms import TextGenerationModelFactory
from prompting_strategies import ZeroShotPromptFactory, FewShotPromptFactory

/Users/detraviousjamaribrinkley/Documents/Development/classes/cai_6307_nlp/comp_express_mt/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Data

In [2]:
file_name = "akuapem_with_tags_dataset-verified_data.xlsx"
path = os.path.join("../data/", file_name)

### Load One to Many

In [3]:
one_to_many_df = pd.read_excel(path, sheet_name="1-M_tags")
akan_source_one = "Akan (Source, One)"
eng_target_many = "English (Target, Many)"
one_many_cols_to_rename = {"Akuapem Twi": akan_source_one, 
                  "English": eng_target_many
                  }
one_to_many_df.rename(columns=one_many_cols_to_rename, inplace=True)
one_to_many_df

,AUD_SIZE,STATUS,AGE,FORMALITY,GENDER,GENDER_2,ANIMACY,SPEECH_ACT,"Akan (Source, One)","English (Target, Many)"
0,INDIVIDUAL,NaN,PEER,INFORMAL,MASCULINE,FEMININE,ANIMATE,STATEMENT,Anadwo biara ɔfrɛ no.,He calls her every night.
1,INDIVIDUAL,NaN,PEER,INFORMAL,MASCULINE,MASCULINE,ANIMATE,STATEMENT,Anadwo biara ɔfrɛ no.,He calls him every night.
2,INDIVIDUAL,NaN,PEER,INFORMAL,FEMININE,FEMININE,ANIMATE,STATEMENT,Anadwo biara ɔfrɛ no.,She calls her every night.
3,INDIVIDUAL,NaN,PEER,INFORMAL,MASCULINE,FEMININE,ANIMATE,STATEMENT,Anadwo biara ɔfrɛ no.,He calls her every night.
4,INDIVIDUAL,NaN,PEER,INFORMAL,FEMININE,MASCULINE,ANIMATE,STATEMENT,"Anɔpa yi, ohyiaa no.",She met him this morning.
...,...,...,...,...,...,...,...,...,...,...
458,INDIVIDUAL,NaN,PEER,FORMAL,NEUTRAL,NEUTRAL,INANIMATE,STATEMENT,"Nokwarem no, osu bɛtɔ.",It is definitely going to rain.
459,INDIVIDUAL,NaN,PEER,FORMAL,NEUTRAL,NEUTRAL,INANIMATE,STATEMENT,"Nokwarem no, osu bɛtɔ.",Rain is surely on the way.
460,SMALL GROUP,NaN,PEER,INFORMAL,NEUTRAL,NEUTRAL,ANIMATE,STATEMENT,Ɛsɛ sɛ yehu akokoaa foforo bi a wɔawo no foforo.,We've got to find a new babysitter.
461,SMALL GROUP,NaN,PEER,INFORMAL,NEUTRAL,NEUTRAL,ANIMATE,STATEMENT,Ɛsɛ sɛ yehu akokoaa foforo bi a wɔawo no foforo.,We need to look for another babysitter.


In [4]:
akan_one_to_eng_many_mappings = one_to_many_df.groupby(akan_source_one)[eng_target_many].apply(list).to_dict()
akan_one_to_eng_many_mappings
# for akan, e_list in akan_one_to_eng_many_mappings.items():
#     print(f"Key: {akan}")
#     print(f"Values: {e_list}\n")

{'Amerika atubrafo a wodi kan no duu hɔ wɔ afeha a ɛto so 17 no mu.': ['The first American colonists arrived in the 17th century.',
  'The first settlers came to America in the 1600s.',
  'America was first colonized in the 17th century.',
  'The first American colonists landed in the 17th century.'],
 'Anadwo biara ɔfrɛ no.': ['He calls her every night.',
  'He calls him every night.',
  'She calls her every night.',
  'He calls her every night.'],
 'Anigyesɛm ne sɛ, hyɛn no mufo biara anhaw.': [' Fortunately, no passengers were injured.',
  ' Luckily, none of the passengers were hurt.',
  ' Thankfully, no passengers were harmed.',
  ' Thankfully, all passengers were safe.'],
 'Anɔpa yi, ohyiaa no.': ['She met him this morning.',
  'She met her this morning.',
  'He met her this morning.',
  'He met him this morning.'],
 'Asamaoh ye nipa kese.': ["Asamoah's great.",
  'Asamoah is fantastic.',
  'Asamoah is wonderful.',
  'Asamoah is an amazing person.'],
 'Asamoah anhu sɛnea na wobu E

In [5]:
akan_sentences = list(akan_one_to_eng_many_mappings.keys())
akan_sentences

['Amerika atubrafo a wodi kan no duu hɔ wɔ afeha a ɛto so 17 no mu.',
 'Anadwo biara ɔfrɛ no.',
 'Anigyesɛm ne sɛ, hyɛn no mufo biara anhaw.',
 'Anɔpa yi, ohyiaa no.',
 'Asamaoh ye nipa kese.',
 'Asamoah anhu sɛnea na wobu Esi anim animtiaa no.',
 'Asamoah betumi abua no.',
 'Asamoah yɛ ɔnokwafo.',
 'Awarefo no de sika pii na ɛyɛɛ wɔn fie.',
 'Bere a ade tɔɔ ne so ara pɛ na yɛde no kɔɔ ayaresabea hɔ.',
 'Bere a Ɔhemmaa no soo nkuruwa no mu no, ɔde guu adaka no mu.',
 'Bere bɛn na wiase no bɛba awiei?',
 'Bɔ mmɔden yɛ saa.',
 'Dɛn na wobɛka sɛ ɛsɛ sɛ wokɔ nnwonto no ase?',
 'Efi saa bere no, minni adagyew koraa.',
 'Egye obi na wahu emu biako.',
 'Esi bɔɔ ne tirim sɛ ɔbɛyɛ ne fie ho mfonini.',
 'Esiane ahum no nti, hyɛn no antumi amfi po so hyɛn gyinabea hɔ.',
 "Esiane sɛ na m'ani so atan me nti, anka mereyɛ akyere me.",
 'Hena na ɔka kyerɛɛ wo sɛ mintumi nnom nsu?',
 'Kwaku anyɛ bɔne biara.',
 'Kwaku dii akɔneaba.',
 'Kwaku ntumi ntu ntɛm sɛnea Abena betumi ayɛ no.',
 "M'ani begye ho s

## Initialize Models + Propmt Models

In [6]:
tgmf = TextGenerationModelFactory
llama_31_70b_instruct = tgmf.create_instance('llama-3.1-70b-instruct')
llama_33_70b_instruct = tgmf.create_instance('llama-3.3-70b-instruct')
models = [llama_31_70b_instruct, llama_33_70b_instruct]
# models = [llama_33_70b_instruct]

In [7]:
def generate_data(prompt: str, model):
    model_output = model.generate(prompt)
    # model_outputs[model.model_name] = model_output
    return int(model_output)

In [8]:
def get_llm_labels(df, model_name):
    filt_llama = (df['llm_name'] == model_name)
    filt_df = df[filt_llama]
    return filt_df['llm_label']

### Few Shot Prompt with Many to One

In [12]:
zero_shot_direct_prompt = ZeroShotPromptFactory("direct")
zero_shot_context_prompt = ZeroShotPromptFactory("context")
few_shot_direct_prompt = FewShotPromptFactory("direct")
few_shot_context_prompt = FewShotPromptFactory("context")
# chain_of_thought_prompt = ChainOfThoughtPrompt
# prompt_names = [zero_shot_prompt, few_shot_prompt, chain_of_thought_prompt]

# prompt_names = [zero_shot_direct_prompt, zero_shot_context_prompt]
prompt_names = [zero_shot_direct_prompt, zero_shot_context_prompt, few_shot_direct_prompt, few_shot_context_prompt]

In [13]:
idx = 0
results = {}
for prompt_name in prompt_names:
    prompt_results = []
    for key, value in tqdm(akan_one_to_eng_many_mappings.items()):
        # print(f"Key: {key}")
        # print(f"\tIndex: {value}")
        
        prompt = prompt_name.get_base_prompt(key, value)
        if idx == 0:
            print(f"Prompt ({prompt_name.get_name()}): {prompt}")
            idx = idx + 1

#         for model in models:
#             llm_result = generate_data(prompt, model)
#             # print(f"Model: {model.__name__()}\tGenerated: {llm_result}\n")
#             result = (key, value, llm_result, f"{prompt_name}_{ model.__name__()}", prompt_name)
#             prompt_results.append(result)
#     results[prompt_name] = prompt_results
    idx = 0
# results

100%|██████████| 132/132 [00:00<00:00, 586491.66it/s]


Prompt (direct): You are translating from Akan to English. Select the most appropriate English translation from the options provided.
        
        Akan sentence: "Amerika atubrafo a wodi kan no duu hɔ wɔ afeha a ɛto so 17 no mu."
        
        Translation options: 
	1. The first American colonists arrived in the 17th century.
	2. The first settlers came to America in the 1600s.
	3. America was first colonized in the 17th century.
	4. The first American colonists landed in the 17th century.
        
        Select the best translation by number only. Respond with just the number (1, 2, 3, etc.).
        


100%|██████████| 132/132 [00:00<00:00, 836326.48it/s]


Prompt (context): You are analyzing an Akan sentence to infer pragmatic context and select the appropriate English translation.
        
        Akan sentence: "Amerika atubrafo a wodi kan no duu hɔ wɔ afeha a ɛto so 17 no mu."
        
        Translation options: 
	1. The first American colonists arrived in the 17th century.
	2. The first settlers came to America in the 1600s.
	3. America was first colonized in the 17th century.
	4. The first American colonists landed in the 17th century.
        
        First, infer the pragmatic context by selecting ONE value for each dimension:
            - Audience: [Individual | Small_Group | Large_Group | Broadcast]
            - Status: [Equal | Superior | Subordinate]
            - Age: [Peer | Elder | Younger]
            - Formality: [Formal | Casual]
            - Gender: [Masculine | Feminine | Neutral]
            - Animacy: [Animate | Inanimate]
            - Speech_Act: [Question | Answer | Statement | Command | Request | Greeting]



100%|██████████| 132/132 [00:00<00:00, 909110.23it/s]


Prompt (direct): You are translating from Akan to English. Select the most appropriate English translation from the options provided.
        Examples:
            Akan: "Ɔyɛ me maame"
            Options: 1. He is my mother 2. She is my mother 3. They are my mother
            Selection: 2

            Akan: "Mema wo akwaaba"
            Options: 1. I welcome you (singular) 2. We welcome you (plural) 3. I welcomed you
            Selection: 1

        Now select for this sentence:
        Akan sentence: "Amerika atubrafo a wodi kan no duu hɔ wɔ afeha a ɛto so 17 no mu."
        
        Translation options: 
	1. The first American colonists arrived in the 17th century.
	2. The first settlers came to America in the 1600s.
	3. America was first colonized in the 17th century.
	4. The first American colonists landed in the 17th century.
        
        Select the best translation by number only. Respond with just the number (1, 2, 3, etc.).
        


100%|██████████| 132/132 [00:00<00:00, 909110.23it/s]

Prompt (context): You are analyzing an Akan sentence to infer pragmatic context and select the appropriate English translation.
        Example 1:
            Akan: "Ɔyɛ me mpena"
            Options: 1. He is my boyfriend 2. She is my girlfriend 3. They are my lover
            Analysis: The term "mpena" suggests romantic relationship (casual register), "Ɔ" is 3rd person singular
            TAGS: Audience=Individual, Status=Equal, Age=Peer, Formality=Casual, Gender=Masculine, Animacy=Animate, Speech_Act=Statement
            SELECTION: 1

            Example 2:
            Akan: "Yɛfrɛ wo sɛn?"
            Options: 1. What is your name? 2. What do they call you? 3. How do we call you?
            Analysis: Direct question format, likely addressing individual, neutral formality
            TAGS: Audience=Individual, Status=Equal, Age=Peer, Formality=Casual, Gender=Neutral, Animacy=Animate, Speech_Act=Question
            SELECTION: 1

        Now select for this sentence:
        Akan

In [11]:
m_to_o_direct_llm_selection

NameError: name 'm_to_o_direct_llm_selection' is not defined

In [ ]:
akan_man_to_eng_one_mappings